In [1]:
%pip install lxml polars


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: /Users/pletcher/code/writing/articles/2024-11-28_tragedy-dfs/.venv/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import polars as pl

df = pl.read_parquet("./greek-tragedy-by-line.parquet")
df

n,urn,dramatist,title,speaker,text,tokens
str,str,str,str,str,str,list[str]
"""1""","""urn:cts:greekLit:tlg0006.tlg01…","""Euripides""","""Ἰφιγένεια ἐν Ταύροις""","""Ἰφιγένεια""","""Πέλοψ ὁ Ταντάλειος ἐς Πῖσαν μο…","[""πελοψ"", ""ο"", … ""μολων""]"
"""2""","""urn:cts:greekLit:tlg0006.tlg01…","""Euripides""","""Ἰφιγένεια ἐν Ταύροις""","""Ἰφιγένεια""","""θοαῖσιν ἵπποις Οἰνομάου γαμεῖ …","[""θοαισιν"", ""ιπποισ"", … ""κορην""]"
"""3""","""urn:cts:greekLit:tlg0006.tlg01…","""Euripides""","""Ἰφιγένεια ἐν Ταύροις""","""Ἰφιγένεια""","""ἐξ ἧς Ἀτρεὺς ἔβλαστεν· Ἀτρέως …","[""εξ"", ""ησ"", … ""παισ""]"
"""4""","""urn:cts:greekLit:tlg0006.tlg01…","""Euripides""","""Ἰφιγένεια ἐν Ταύροις""","""Ἰφιγένεια""","""Μενέλαος Ἀγαμέμνων τε· τοῦ δʼ …","[""μενελαοσ"", ""αγαμεμνων"", … ""εγω""]"
"""5""","""urn:cts:greekLit:tlg0006.tlg01…","""Euripides""","""Ἰφιγένεια ἐν Ταύροις""","""Ἰφιγένεια""","""τῆς Τυνδαρείας θυγατρὸς Ἰφιγέν…","[""τησ"", ""τυνδαρειασ"", … ""παισ""]"
…,…,…,…,…,…,…
"""1526""","""urn:cts:greekLit:tlg0011.tlg00…","""Sophocles""","""Οἰδίπους Τύραννος""","""Χορός""","""οὗ τίς οὐ ζήλῳ πολιτῶν ἦν τύχα…","[""ου"", ""τισ"", … ""επιβλεπων""]"
"""1527""","""urn:cts:greekLit:tlg0011.tlg00…","""Sophocles""","""Οἰδίπους Τύραννος""","""Χορός""","""εἰς ὅσον κλύδωνα δεινῆς συμφορ…","[""εισ"", ""οσον"", … ""εληλυθεν""]"
"""1528""","""urn:cts:greekLit:tlg0011.tlg00…","""Sophocles""","""Οἰδίπους Τύραννος""","""Χορός""","""ὥστε θνητὸν ὄντα κείνην τὴν τε…","[""ωστε"", ""θνητον"", … ""ιδειν""]"


In [5]:
with pl.Config(tbl_rows=-1):
    res = (
        df.filter(pl.col("title") != "Ichneutae")
        .group_by("dramatist", "title")
        .agg(pl.col("tokens").explode())
        .select(
            "dramatist",
            "title",
            n_types=pl.col("tokens").list.n_unique(),
            n_tokens=pl.col("tokens").list.len(),
            ttr=(pl.col("tokens").list.n_unique() / pl.col("tokens").list.len()),
        )
        .sort("ttr")
    )

    print(res)

    avgs = (
        res.group_by("dramatist")
        .agg(pl.col("ttr").mean().alias("avg_ttr"))
        .select("dramatist", "avg_ttr")
        .sort("avg_ttr")
    )

    print(avgs)

shape: (33, 5)
┌───────────┬──────────────────────┬─────────┬──────────┬──────────┐
│ dramatist ┆ title                ┆ n_types ┆ n_tokens ┆ ttr      │
│ ---       ┆ ---                  ┆ ---     ┆ ---      ┆ ---      │
│ str       ┆ str                  ┆ u32     ┆ u32      ┆ f64      │
╞═══════════╪══════════════════════╪═════════╪══════════╪══════════╡
│ Sophocles ┆ Οἰδίπους Τύραννος    ┆ 3694    ┆ 9293     ┆ 0.397503 │
│ Sophocles ┆ Οἰδίπους ἐπὶ Κολωνῷ  ┆ 4158    ┆ 10398    ┆ 0.399885 │
│ Sophocles ┆ Φιλοκτήτης           ┆ 3537    ┆ 8824     ┆ 0.400839 │
│ Sophocles ┆ Ἠλέκτρα              ┆ 3628    ┆ 8700     ┆ 0.417011 │
│ Euripides ┆ Ὀρέστης              ┆ 4178    ┆ 9865     ┆ 0.423517 │
│ Euripides ┆ Ἰφιγένεια ἐν Αὐλίδι  ┆ 3951    ┆ 9205     ┆ 0.429223 │
│ Euripides ┆ Ἑλένη                ┆ 4223    ┆ 9787     ┆ 0.431491 │
│ Euripides ┆ Ἴων                  ┆ 4027    ┆ 9219     ┆ 0.436815 │
│ Euripides ┆ Ἡρακλεῖδαι           ┆ 2737    ┆ 6160     ┆ 0.444318 │
│ Euripides ┆ Μήδει